In [1]:

functions = [
    {
        "name": "check_free_slots",
        "description": "Check available meeting time slots on a specific date.",
        "parameters": {
            "type": "object",
            "properties": {
                "date": {
                    "type": "string",
                    "description": "Date to check availability. Format: YYYY-MM-DD"
                },
                "duration_minutes": {
                    "type": "integer",
                    "description": "Duration of the meeting in minutes. Default is 60.",
                    "default": 60
                }
            },
            "required": ["date"]
        }
    },
    {
        "name": "schedule_meeting",
        "description": "Schedule a meeting in the calendar.",
        "parameters": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": "Title or summary of the meeting."
                },
                "start_time": {
                    "type": "string",
                    "description": "Start time in ISO 8601 format (e.g., 2025-06-20T14:00:00)"
                },
                "end_time": {
                    "type": "string",
                    "description": "End time in ISO 8601 format (e.g., 2025-06-20T15:00:00)"
                },
                "attendees_emails": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "format": "email"
                    },
                    "description": "List of attendee email addresses."
                }
            },
            "required": ["summary", "start_time", "end_time"]
        }
    },
    {
        "name": "cancel_meeting",
        "description": "Cancel a scheduled calendar event by exact date and start time.",
        "parameters": {
            "type": "object",
            "properties": {
                "date_str": {
                    "type": "string",
                    "description": "Date of the event. Format: YYYY-MM-DD"
                },
                "time_str": {
                    "type": "string",
                    "description": "Time of the event. Format: HH:MM in 24-hour format"
                }
            },
            "required": ["date_str", "time_str"]
        }
    },
    {
        "name": "list_upcoming_meeting",
        "description": "List events scheduled for a specific date, optionally filtered by time.",
        "parameters": {
            "type": "object",
            "properties": {
                "date_str": {
                    "type": "string",
                    "description": "Date to fetch events from. Format: YYYY-MM-DD"
                },
                "time_str": {
                    "type": "string",
                    "description": "Optional time filter. Format: HH:MM in 24-hour format"
                }
            },
            "required": ["date_str"]
        }
    }
]


In [17]:
import openai
import json
import getpass
import os
from functions import functions
from calendar_code import check_free_slots, schedule_meeting, cancel_meeting, list_upcoming_meeting
from datetime import datetime

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# 🧠 Memory buffer (chat history)
now = datetime.now()
current_date = now.strftime("%Y-%m-%d")
current_time = now.strftime("%H:%M")


system_prompt = f"""
You are Smart Scheduler, a voice-based AI assistant that helps users manage their Google Calendar.
You assist with: Scheduling new meetings,Checking available slots,Listing meetings,Canceling meetings
Today's date is {current_date}, and current time is {current_time} — use this for all "today", "now", or "tomorrow" references.
Engage in multi-turn, context-aware conversations. Always clarify missing info like:
Meeting date, time, duration, Title (summary),Attendees emails , Cancellation confirmations or scheduling conflicts

Handle complex queries:
Deadlines (e.g., “before my flight on Friday at 6 PM”)
Relative dates (e.g., “2 days after Project Kickoff”)
Vague time (“evening”, “usual sync-up”)
Conflicts (suggest alternatives if slot is taken)

When all details are available, call the appropriate function silently.
Never guess — ask if unsure.
Respond in short human way maximum 2-3 sentences, natural, and warm assistant-style replies — ideal for voice (TTS). Examples:
“Got it. Let me check.”
“Just a moment.”
“Would you like to invite anyone else?”
“Here are the options.”
Be precise with date/time formats and always summarize outcomes clearly.
"""

function_map = {
    "check_free_slots": check_free_slots,
    "schedule_meeting": schedule_meeting,
    "cancel_meeting": cancel_meeting,
    "list_upcoming_meeting": list_upcoming_meeting,
}

def api_call(chat_history):
        response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=chat_history,
        functions=functions,
        function_call="auto",
        stream= True
        )

        for chunk in response: 
            print(f"this is api call {chunk}")
            yield chunk                                 # hand each chunk back to caller

def api_call_function(message,chat_history):
        func_name = message.function_call.name
        args = json.loads(message.function_call.arguments)
           
         # Dynamically call the function
        if func_name in function_map:
            result = function_map[func_name](**args)
        else:
            raise ValueError(f"Unknown function: {func_name}")

        # Append function response to memory
        chat_history.append({
                "role": "function",
                "name": func_name,
                "content": json.dumps(result)
        })

            # step 2: Call GPT again with tool output
        second_response = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=chat_history
        )
        final_reply = second_response.choices[0].message.content
        return final_reply  




In [3]:
chat_history = [{"role": "system", "content": system_prompt}]


In [20]:
message = "I want to schedule a meeting today at 10 am for 1 hour and dont want to invite anyone. The title of meeting will be team meeting"
buf=""
chat_history.append({"role": "user", "content": message})
for chunk in api_call(chat_history):
        print (f"This is chunk {chunk}")
        delta = chunk.choices[0].delta
        print (f"This is delta {delta}")
        buf += delta.content or ""
        # Flush on punctuation or ≥120 chars
        if any(buf.endswith(p) for p in ".!?") or len(buf) >= 120:
            print("This is buf")
            print(buf)

            

this is api call ChatCompletionChunk(id='chatcmpl-Bo3NivYAxl49cwuESgmVC8sdWmkWK', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1751269098, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier='default', system_fingerprint='fp_62a23a81ef', usage=None)
This is chunk ChatCompletionChunk(id='chatcmpl-Bo3NivYAxl49cwuESgmVC8sdWmkWK', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1751269098, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier='default', system_fingerprint='fp_62a23a81ef', usage=None)
This is delta ChoiceDelta(content='', function_call=None, refusal=None, role='assistant', tool_calls=None)
this is api call ChatCompletionChunk(id='chatcmpl-Bo3NivYAxl49cwuESgmVC8sdWmkWK', choices=[Choic

In [ ]:
message = "check free slots tomorrow morning for 1 hour"
buf=""
fn_name        = None
fn_args_raw    = ""       # JSON comes in fragments!

chat_history.append({"role": "user", "content": message})
for chunk in api_call(chat_history):
        delta = chunk.choices[0].delta
   
        if delta.function_call:
            print("delta")
            print(delta.function_call)
            if delta.function_call.name:
                print("fucntion")
                fn_name = delta.function_call.name      # appears once
            if delta.function_call.arguments:
                print("arguments")
                fn_args_raw += delta.function_call.arguments  # append fragment
                print(fn_args_raw)

        else:
            print (f"This is chunk {chunk}")
            print (f"This is delta {delta}")
            buf += delta.content or ""
            # Flush on punctuation or ≥120 chars
            if any(buf.endswith(p) for p in ".!?") or len(buf) >= 120:
                print("This is buf")
                print(buf)
print("done")
print(fn_name)
print(fn_args_raw)
type

this is api call ChatCompletionChunk(id='chatcmpl-Bo3WhqiGd8Zi07IggCbKvCl9V46eD', choices=[Choice(delta=ChoiceDelta(content=None, function_call=ChoiceDeltaFunctionCall(arguments='', name='check_free_slots'), refusal=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1751269655, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier='default', system_fingerprint='fp_62a23a81ef', usage=None)
delta
ChoiceDeltaFunctionCall(arguments='', name='check_free_slots')
fucntion
this is api call ChatCompletionChunk(id='chatcmpl-Bo3WhqiGd8Zi07IggCbKvCl9V46eD', choices=[Choice(delta=ChoiceDelta(content=None, function_call=ChoiceDeltaFunctionCall(arguments='{"', name=None), refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1751269655, model='gpt-4o-mini-2024-07-18', object='chat.completion.chunk', service_tier='default', system_fingerprint='fp_62a23a81ef', usage=None)
delta
ChoiceDeltaFunc

In [40]:
type(fn_args_raw)


dict

In [39]:
fn_args_raw = json.loads(fn_args_raw)
